In [2]:
import pandas as pd
import keras.losses
import keras.metrics
keras.losses.loss = keras.losses.mse
keras.metrics.loss = keras.metrics.mse
import json
from video_process_utils import *
from keras.models import load_model
from statsmodels.regression.linear_model import OLSResults
import csv
import numpy as np

In [3]:
file = "16315504-processed"

In [4]:
path = "D:/OneDrive - Anna University/project/dataset/video-gait/openpose/%s/" %\
    (str(file))

In [5]:
def extract(path):
    num_p = ""
    for c in path:
        if c.isdigit():
            num_p = num_p + c
    return num_p

In [6]:
video_ID = extract(path)
video_ID

'16315504'

In [7]:
def convert_json2csv(json_dir):
    resL = np.zeros((2000,75))
    resL[:] = np.nan
    for frame in range(1,2000):
        temp_dir = extract(json_dir)
        test_image_json = json_dir+temp_dir+'-processed_%s_keypoints.json' %\
            (str(frame).zfill(12)) #input_000000000000_keypoints.json

        if not os.path.isfile(test_image_json):
            break
        with open(test_image_json) as data_file:  
            data = json.load(data_file)

        for person in data['people']:
            keypoints = person['pose_keypoints_2d']
            xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
            counter = 0
            resL[frame-1,:] = keypoints
            break
    check = np.apply_along_axis(lambda x: np.any(~np.isnan(x)),1,resL)
    for i in range(len(check)-1,-1,-1):
        if check[i]:
            break
    return resL[:i+1]

In [8]:
frames = convert_json2csv(path)
frames = frames[~np.isnan(frames).any(axis=1)]
# pd.DataFrame(frames).to_csv('keypoints.csv',index=False)
pd.DataFrame(frames)

,0,1,2,3,4,5,6,7,8,9,...,65,66,67,68,69,70,71,72,73,74
0,496.523,120.649,0.931581,505.535,140.081,0.917137,487.321,137.569,0.855514,476.875,...,0.627137,495.186,277.231,0.868053,488.617,277.184,0.814118,496.412,273.280,0.737735
1,496.503,120.643,0.941526,504.309,140.103,0.915157,487.310,138.832,0.870269,476.877,...,0.621181,495.181,277.223,0.876888,488.612,277.169,0.821110,495.217,271.953,0.712627
2,496.440,120.626,0.966148,505.528,138.875,0.903764,487.366,138.811,0.844444,476.924,...,0.627729,495.195,277.243,0.858960,488.624,277.199,0.814462,495.179,271.999,0.732808
3,496.430,120.553,0.981796,505.536,138.856,0.906283,487.370,137.561,0.838636,476.933,...,0.635278,495.190,277.215,0.881837,488.627,277.164,0.817597,495.203,270.694,0.711248
4,496.417,120.524,0.972635,505.542,138.804,0.894608,487.345,137.512,0.840324,476.921,...,0.643878,495.154,277.176,0.872772,487.363,275.923,0.809180,495.191,270.684,0.693193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,463.899,108.828,0.953406,474.208,112.759,0.858828,454.663,115.359,0.831974,453.377,...,0.585150,483.408,290.258,0.741882,474.307,288.938,0.745628,484.677,275.870,0.684265
595,465.083,108.830,0.955394,474.208,112.768,0.858094,454.654,115.375,0.832429,453.353,...,0.577059,483.409,290.244,0.729042,474.315,288.928,0.731578,484.669,275.852,0.668060
596,465.084,108.829,0.955666,472.981,112.750,0.855546,453.430,115.345,0.837056,452.116,...,0.589999,483.403,290.233,0.716449,474.311,287.701,0.717175,484.656,274.607,0.649843
597,465.092,108.833,0.957745,473.000,112.775,0.850574,453.437,115.349,0.834558,452.115,...,0.589536,483.404,290.236,0.718937,474.313,287.700,0.723696,484.672,275.843,0.663053


In [9]:
centered_filtered = process_video_and_add_cols(frames)
centered_filtered_noswap = process_video_and_add_cols(frames,
                                swap_orientation=False)

In [10]:
def get_prediction(centered_filtered, col, side = None):
    model = load_model("models/{}_best.pb".format(col))
    correction_model = OLSResults.load("models/{}_correction.pb".format(col))
    maps = {
        "GDI": (25, 100)
    }

    def undo_scaling(y,target_min,target_range):
        return y*target_range+target_min

    preds = []

    video_len = centered_filtered.shape[0]
    
    cols = x_columns
    if side == "L":
        cols = x_columns_left
    if side == "R":
        cols = x_columns_right

    samples = []
    for nstart in range(0,video_len-124,31):
        samples.append(centered_filtered[nstart:(nstart+124),cols])
    X = np.stack(samples)
    
    p = model.predict(X)[:,0]
    p = undo_scaling(p, maps[col][0], maps[col][1])
    p = np.transpose(np.vstack([p,np.ones(p.shape[0])]))
    p = correction_model.predict(pd.DataFrame(p))
    gdi = np.mean(p) 
    pred = res(gdi)
    return pred

0 - No Risk
1 - Risk Found

In [13]:
pred = get_prediction(centered_filtered, "GDI")
pred = str(pred)
pred

'1'

In [ ]:
path = "D:/OneDrive - Anna University/project/dataset/video-gait/openpose/"
subfolders = [f.path for f in os.scandir(path) if f.is_dir()]

for subfolder in subfolders:
    path = (subfolder + '/')
    def extract(path):
        num_p = ""
        for c in path:
            if c.isdigit():
                num_p = num_p + c
        return num_p
    video_ID = extract(path)
    if(video_ID > '14827902'):
        def convert_json2csv(json_dir):
            resL = np.zeros((2000,75))
            resL[:] = np.nan
            for frame in range(1,2000):
                temp_dir = extract(json_dir)
                test_image_json = json_dir+temp_dir+'-processed_%s_keypoints.json' %\
                    (str(frame).zfill(12)) #input_000000000000_keypoints.json

                if not os.path.isfile(test_image_json):
                    break
                with open(test_image_json) as data_file:  
                    data = json.load(data_file)

                for person in data['people']:
                    keypoints = person['pose_keypoints_2d']
                    xcoords = [keypoints[i] for i in range(len(keypoints)) if i % 3 == 0]
                    counter = 0
                    resL[frame-1,:] = keypoints
                    break
            check = np.apply_along_axis(lambda x: np.any(~np.isnan(x)),1,resL)
            for i in range(len(check)-1,-1,-1):
                if check[i]:
                    break
            return resL[:i+1]   


        frames = convert_json2csv(path)
        frames = frames[~np.isnan(frames).any(axis=1)]
        centered_filtered = process_video_and_add_cols(frames)
        centered_filtered_noswap = process_video_and_add_cols(frames,
                                        swap_orientation=False)


        
        ankle_knee_hip_leftangle = get_angle(LANK, LKNE, LHIP, centered_filtered)
        ankle_knee_hip_rightangle = get_angle(RANK, RKNE, RHIP, centered_filtered)
        toe_ankle_knee_leftangle = get_angle(LBTO, LANK, LKNE, centered_filtered)
        toe_ankle_knee_rightangle = get_angle(RBTO, RANK, RKNE, centered_filtered)
        toe_ankle_dist_left = get_distance(LBTO, LANK, centered_filtered)
        toe_ankle_dist_right = get_distance(RBTO, RANK, centered_filtered)
        lank_rank_xdist = centered_filtered[:, 2 * RANK] - centered_filtered[:, 2 * LANK]
        rank_lank_xdist = centered_filtered[:, 2 * LANK] - centered_filtered[:, 2 * RANK]
        # get mean value of ankle_knee_hip_leftangle
        ankle_knee_hip_leftangle_mean = np.mean(ankle_knee_hip_leftangle).astype(str)
        ankle_knee_hip_rightangle_mean = np.mean(ankle_knee_hip_rightangle).astype(str)
        toe_ankle_knee_leftangle_mean = np.mean(toe_ankle_knee_leftangle).astype(str)
        toe_ankle_knee_rightangle_mean = np.mean(toe_ankle_knee_rightangle).astype(str)
        toe_ankle_dist_left_mean = np.mean(toe_ankle_dist_left).astype(str)
        toe_ankle_dist_right_mean = np.mean(toe_ankle_dist_right).astype(str)
        lank_rank_xdist_mean = np.mean(lank_rank_xdist).astype(str)
        rank_lank_xdist_mean = np.mean(rank_lank_xdist).astype(str)


        def get_prediction(centered_filtered, col, side = None):
            model = load_model("models/{}_best.pb".format(col))
            correction_model = OLSResults.load("models/{}_correction.pb".format(col))
            maps = {
                "GDI": (25, 100)
            }

            def undo_scaling(y,target_min,target_range):
                return y*target_range+target_min

            preds = []

            video_len = centered_filtered.shape[0]
            
            cols = x_columns
            if side == "L":
                cols = x_columns_left
            if side == "R":
                cols = x_columns_right

            samples = []
            for nstart in range(0,video_len-124,31):
                samples.append(centered_filtered[nstart:(nstart+124),cols])
            X = np.stack(samples)
            
            p = model.predict(X)[:,0]
            p = undo_scaling(p, maps[col][0], maps[col][1])
            p = np.transpose(np.vstack([p,np.ones(p.shape[0])]))
            p = correction_model.predict(pd.DataFrame(p))
            gdi = np.mean(p) 
            pred = res(gdi)
            return pred


        pred = get_prediction(centered_filtered, "GDI")
        pred = str(pred)
        print(video_ID)
        with open('prediction_1.csv', 'a+') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([video_ID,
                            ankle_knee_hip_leftangle_mean,
                            ankle_knee_hip_rightangle_mean,
                            toe_ankle_knee_leftangle_mean,
                            toe_ankle_knee_rightangle_mean,
                            toe_ankle_dist_left_mean,
                            toe_ankle_dist_right_mean,
                            lank_rank_xdist_mean,
                            rank_lank_xdist_mean,
                            pred])
    else:
        print("skipping")
